In [2]:
import numpy as np

import astropy.io.fits as fits

import glob

# Holy shit it worked.

from lsst.sims.photUtils.Sed import Sed

In [3]:
dir = '/Users/ichorgodbel-shamroth/lsst_stack/fits_files/ck04models/ckm05/'
filepath = glob.glob(dir+'*.fits')

In [4]:
for filename in filepath:
    spectra_data = fits.getdata(filename)

In [5]:
# Progress!

for filename in filepath:
    spectrum = fits.open(filename)

In [6]:
for filename in filepath:
    sed_spec = Sed(filename)
    #print(sed_spec)

In [7]:
test_SED = Sed(dir+'ckm05_3500.fits')

In [8]:
Sed?